# Introduction

In this notebook we demonstrate the use of **Word Embeddings (Word2Vec)** weighting technique into Information Retrieval to make trace link recovery between Use Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each use case content as an entire document that must be returned to the query made

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import spacy

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support, pairwise_distances, pairwise
from sklearn.externals.joblib import Parallel, delayed
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import Normalizer

from scipy.sparse import csr_matrix

import nltk
import datetime
import pprint
from enum import Enum
import pickle
from tqdm import tqdm

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import warnings; warnings.simplefilter('ignore')

### Oracle Loader

In [2]:
class OracleLoader:
    def __init__(self, rows_names, columns_names):
        self.oracle = None
        self._columns_names = columns_names
        self._rows_names = rows_names
    
    def load(self):
        self.oracle = pd.DataFrame(columns=list(self._columns_names), 
                                   data=np.zeros(shape=(len(self._rows_names), len(self._columns_names)), 
                                                 dtype='int64'))
        self.oracle.insert(0, 'artf_name', list(self._rows_names))
        
        for index, row in trace_df.iterrows():
            idx = self.oracle[self.oracle.artf_name == row['trg_artf']].index
            self.oracle.at[idx, row['src_artf']] = row['link']

        self.oracle.set_index('artf_name', inplace=True)

## Load Dataset and Preprocessing

In [3]:
trace_df = pd.read_csv('../../data/jEdit/jEditDataset/oracle/output/trace_matrix.csv')
artfs_desc_df = pd.read_csv('../../data/jEdit/jEditDataset/oracle/output/artifacts_descriptions.csv', sep="|")

use_cases_df = artfs_desc_df[artfs_desc_df.artf_description.str.contains('Use Case ID')]
bug_reports_df = artfs_desc_df[artfs_desc_df.artf_description.str.contains('Bug Number')]

corpus = use_cases_df.artf_description
query = bug_reports_df.artf_description

use_cases_names = use_cases_df.artf_name
bug_reports_names = bug_reports_df.artf_name

orc = OracleLoader(use_cases_names, bug_reports_names)
orc.load()

# WordVec Based Model

#### Model Hyperparameters

In [4]:
class WordVec_Model_Hyperp(Enum):
    NAME = 'wordvec__name'
    TOP = 'wordvec__top'
    TOKENIZER = 'wordvec__tokenizer'
    SIM_MEASURE_MIN_THRESHOLD = 'wordvec__sim_measure_min_threshold'

#### Tokenizers

In [5]:
"""
Others stemmers are not relevant for our analysis:
 . RSLP Stemmer: portuguese language
 . ISRIS Stemmer: returns Arabic root for the given token 
 . Regexp Stemmer: uses regulax expressions to identify morphological affixes
 
Relevant Stemmers/Lemmatizers are implemented below. 
"""

class GenericTokenizer(object):
    def __init__(self):
        self.stopwords = nltk.corpus.stopwords.words('english')
    def __call__(self, doc):
        tokens = [self.stemmer.stem(token) for token in nltk.word_tokenize(doc)]
        #return [token.lower() for token in tokens if token.isalpha() and token not in self.stopwords and len(token) > 1]
        return [token.lower() for token in tokens if token not in self.stopwords]
        
class WordNetBased_LemmaTokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.wnl = nltk.stem.WordNetLemmatizer()
    def __call__(self, doc):
        tokens = [self.wnl.lemmatize(token) for token in nltk.word_tokenize(doc)]
        return [token.lower() for token in tokens if token.isalpha() and token not in self.stopwords]

class LancasterStemmerBased_Tokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.LancasterStemmer()
    def __call__(self, doc):
        return super().__call__(doc)

class PorterStemmerBased_Tokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.PorterStemmer()
    def __call__(self, doc):
        return super().__call__(doc)
    
class SnowballStemmerBased_Tokenizer(GenericTokenizer):    
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.SnowballStemmer('english')    
    def __call__(self, doc):
        return super().__call__(doc)
        

#### Distance Functions / Similarity Measures Available

In [6]:
class SimilarityMeasure(Enum):
    COSINE = 'cosine'

#### Model Definition

In [12]:
"""
params_dict = {
    'wordvec__sim_measure_min_threshold' : ('cosine',.9),
    'wordvec__name' : 'WordVec',
    'wordvec__top' : 3
    'wordvec_tokenizer' : WordNetBased_LemmaTokenizer()
}
"""
class WordVec_BasedModel:
    def __init__(self, **kwargs):
        """
            Each spaCy NLP model is a 
        """
        self._nlp_model = None
        
        self.name = None
        self.sim_measure_min_threshold = None
        self.top = None
        self.trace_links_df = None
        self.tokenizer = None
        
        self.set_basic_params(**kwargs)
        self.set_nlp_model()
    
    def set_name(self, name):
        self.name = name
    
    def set_basic_params(self, **kwargs):
        self.name = 'WordVec' if WordVec_Model_Hyperp.NAME.value not in kwargs.keys() else kwargs[WordVec_Model_Hyperp.NAME.value]
        self.sim_measure_min_threshold = (SimilarityMeasure.COSINE.value,.80) if WordVec_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value not in kwargs.keys() else kwargs[WordVec_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value]
        self.top = 3 if WordVec_Model_Hyperp.TOP.value not in kwargs.keys() else kwargs[WordVec_Model_Hyperp.TOP.value]
        self.tokenizer = WordNetBased_LemmaTokenizer() if WordVec_Model_Hyperp.TOKENIZER.value not in kwargs.keys() else kwargs[WordVec_Model_Hyperp.TOKENIZER.value]
    
    def set_nlp_model(self):
        """
            WordVec based on GloVe 1.1M keys x 300 dim
            300-dimensional word vectors trained on Common Crawl with GloVe.
        """
        self._nlp_model = spacy.load('en_vectors_web_lg')
    
    def __getstate__(self):
        """to pickle object serialization/deserialization"""
        d = dict(self.__dict__)
        del d['_nlp_model']
        return d
    
    def __setstate__(self, d):
        """to pickle object serialization/deserialization"""
        self.__dict__.update(d)
    
    def recover_links(self, corpus, query, use_cases_names, bug_reports_names):
        return self._recover_links_cosine(corpus, query, use_cases_names, bug_reports_names)
    
    def _recover_links_cosine(self, corpus, query, use_cases_names, bug_reports_names):
        list_corpus_tokens = [self.tokenizer.__call__(doc) for doc in corpus]
        list_query_tokens = [self.tokenizer.__call__(doc) for doc in query]
        
        corpus = [' '.join(tok_list) for tok_list in list_corpus_tokens]
        query = [' '.join(tok_list) for tok_list in list_query_tokens]
        
        self._sim_matrix = pd.DataFrame(index = use_cases_names, 
                                           columns = bug_reports_names,
                                           data=np.zeros(shape=(len(use_cases_names), len(bug_reports_names)),dtype='float64'))
        
        for bug_id, bug_desc in zip(bug_reports_names, query):
            for uc_id, uc_desc in zip(use_cases_names, corpus):
                doc1 = self._nlp_model(bug_desc)
                doc2 = self._nlp_model(uc_desc)
                self._sim_matrix.at[uc_id, bug_id] = doc1.similarity(doc2)  # cosine similarity is default
        
        self._sim_matrix = pd.DataFrame(self._sim_matrix, index=use_cases_names, columns=bug_reports_names)
        self._fillUp_traceLinksDf(use_cases_names, bug_reports_names, self._sim_matrix)        
    
    def _fillUp_traceLinksDf(self, use_cases_names, bug_reports_names, sim_matrix):
        self.trace_links_df = pd.DataFrame(index = use_cases_names,
                                           columns = bug_reports_names,
                                           data = sim_matrix.values)
        
        for col in self.trace_links_df.columns:
            nlargest_df = self.trace_links_df.nlargest(n = self.top, columns=col, keep='first')    
            self.trace_links_df[col] = [1 if x in nlargest_df[col].tolist() and x >= self.sim_measure_min_threshold[1] else 0 for x in self.trace_links_df[col]]
    
    def model_setup(self):
        return {"Setup" : 
                  [
                      {"Name" : self.name},
                      {"Similarity Measure and Minimum Threshold" : self.sim_measure_min_threshold},
                      {"Top Value" : self.top},
                      {"Tokenizer" : self.tokenizer}
                  ]
               }
    
    def get_name(self):
        return self.name
    
    def get_top_value(self):
        return self.top
    
    def get_sim_matrix(self):
        return self._sim_matrix
    
    def get_sim_measure_min_threshold(self):
        return self.sim_measure_min_threshold
    
    def get_trace_links_df(self):
        return self.trace_links_df
    
    def get_tokenizer_type(self):
        return type(self.tokenizer)
    
    def save_sim_matrix(self):
        self._sim_matrix.to_csv('../best_models_sim_matrix/wv.csv')

### Model Evaluator

In [8]:
class ModelEvaluator:
    def __init__(self, oracle, model):
        self.model = model
        self.oracle = oracle
        self.recovered_links = model.trace_links_df
        
        self.eval_df = pd.DataFrame(columns=['precision','recall','fscore','support'])
        self.mean_precision = -1
        self.mean_recall = -1
        self.mean_fscore = -1
    
    def evaluate_model(self, verbose=False, file=None):
        y_true = csr_matrix(self.oracle.values, dtype=int)
        y_pred = csr_matrix(self.recovered_links.values, dtype=int)
        
        p, r, f, sp = precision_recall_fscore_support(y_true, y_pred)

        i = 0
        for idx, row in self.oracle.iteritems():
            self.eval_df.at[idx, 'precision'] = p[i]
            self.eval_df.at[idx, 'recall'] = r[i]
            self.eval_df.at[idx, 'fscore'] = f[i]
            self.eval_df.at[idx, 'support'] = sp[i]
            i += 1
        
        self.mean_precision = self.eval_df.precision.mean()
        self.mean_recall = self.eval_df.recall.mean()
        self.mean_fscore = self.eval_df.fscore.mean()
        
        if verbose:
            self.print_report(file)
    
    #def check_best_model(self, best_pre, best_rec, best_fs, best_md):
    #    if best_rec <= self.get_mean_recall():
    #        if best_pre <= self.get_mean_precision():
    #            return (self.get_mean_precision(), self.get_mean_recall(), self.get_mean_fscore(), self.get_model())
    #    return (best_pre, best_rec, best_fs, best_md)
    
    def print_report(self, file=None):
        dic = self.model.model_setup()
        dic['Measures'] = {}
        dic['Measures']['Mean Precision of {}'.format(self.model.get_name())] = self.get_mean_precision()
        dic['Measures']['Mean Recall of {}'.format(self.model.get_name())] = self.get_mean_recall()
        dic['Measures']['Mean FScore of {}'.format(self.model.get_name())] = self.get_mean_fscore()
        
        if file is None:    
            pprint.pprint(dic)
        else:
            file.write(pprint.pformat(dic))
        
    def plot_precision_vs_recall(self):
        plt.figure(figsize=(6,6))
        plt.plot(self.eval_df.recall, self.eval_df.precision, 'ro', label='Precision vs Recall')

        plt.ylabel('Precision')
        plt.xlabel('Recall')

        plt.axis([0, 1.1, 0, 1.1])
        plt.title("Precision vs Recall Plot - " + self.model.get_name())
        plt.show()
    
    def save_log(self):
        print("\nSaving model log...")
        with open('../logs/' + str(datetime.datetime.now()) + '.txt', 'a') as f:
            evaluator.evaluate_model(verbose=True, file=f)
        print("Model log saved with success!")
            
    def get_mean_precision(self):
        return self.mean_precision
    
    def get_mean_recall(self):
        return self.mean_recall
    
    def get_mean_fscore(self):
        return self.mean_fscore

    def get_model(self):
        return self.model

## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of WordVec Model

In [9]:
best_model = WordVec_BasedModel()
best_model.recover_links(corpus, query, use_cases_names, bug_reports_names)
evaluator = ModelEvaluator(orc.oracle, best_model)
evaluator.evaluate_model(verbose=True)

{'Measures': {'Mean FScore of WordVec': 0.2714285714285714,
              'Mean Precision of WordVec': 0.19047619047619047,
              'Mean Recall of WordVec': 0.5},
 'Setup': [{'Name': 'WordVec'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 3},
           {'Tokenizer': <__main__.WordNetBased_LemmaTokenizer object at 0x7fef689b4160>}]}


### Auxiliary Functions

In [10]:
from itertools import product

def generate_params_comb_list(**kwargs):
    list_params = []
    for key, values in kwargs.items():
        aux_list = []
        for v in values:
            aux_list.append((key, v))
        list_params.append(aux_list)
    
    list_tuples = list(product(*list_params))
    
    list_dicts = []
    for ex_tup in list_tuples:
        dic = {}
        for in_tup in ex_tup:
            dic[in_tup[0]] = in_tup[1]
        list_dicts.append(dic)
        
    return list_dicts


def plot_heatmap(results_df):
    tmp_df = pd.DataFrame({'precision': results_df['precision'], 
                           'recall' : results_df['recall'], 
                           'fscore': results_df['fscore'], 
                           'model': results_df['model_name']})
    tmp_df.set_index('model', inplace=True)
    fig, ax = plt.subplots(figsize=(10, 20)) 
    ax = sns.heatmap(tmp_df, vmin=0, vmax=1, linewidths=.5, cmap="Greens", annot=True, cbar=False, ax=ax)


def highlight_df(df):
    cm = sns.light_palette("green", as_cmap=True)
    return df.style.background_gradient(cmap=cm)    

### Find The Best Model

In [13]:
all_hyperparams = {
    WordVec_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.75,.85,.95]],
    WordVec_Model_Hyperp.TOP.value : [3,5],
    WordVec_Model_Hyperp.TOKENIZER.value : [PorterStemmerBased_Tokenizer(), LancasterStemmerBased_Tokenizer(), 
                                                   WordNetBased_LemmaTokenizer(), SnowballStemmerBased_Tokenizer()]
}

hyperparams = generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

results_df = pd.DataFrame(columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])

#def run_model(idx, **hyperp):    
for idx,hp in tqdm(enumerate(hyperparams)):
    current_model = WordVec_BasedModel(**hp)
    current_model.set_name('WordVec_Based_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, use_cases_names, bug_reports_names)
    
    evaluator = ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    
    model_dump = 'dumps/model/{}.p'.format(evaluator.get_model().get_name())
    evaluator_dump = 'dumps/evaluator/eval_{}.p'.format(evaluator.get_model().get_name())
    
    pickle.dump(evaluator.get_model(), open(model_dump, 'wb'))
    pickle.dump(evaluator, open(evaluator_dump, 'wb'))
    
    results_df = results_df.append(pd.DataFrame([[evaluator.get_mean_precision(), 
                    evaluator.get_mean_recall(),
                    evaluator.get_mean_fscore(), 
                    evaluator.get_model().get_name(),
                    evaluator.get_model().get_top_value(),
                    evaluator.get_model().get_tokenizer_type(),
                    evaluator.get_model().get_sim_measure_min_threshold()[0],
                    evaluator.get_model().get_sim_measure_min_threshold()[1],
                    model_dump,
                    evaluator_dump
           ]], columns=results_df.columns), ignore_index=True)

#tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str, 'top_value': int})


0it [00:00, ?it/s]

Performing model hyperparameters search...



1it [00:10, 10.01s/it]
2it [00:19,  9.94s/it]
3it [00:29,  9.91s/it]
4it [00:39,  9.87s/it]
5it [00:49,  9.85s/it]
6it [00:58,  9.81s/it]
7it [01:08,  9.86s/it]
8it [01:19,  9.94s/it]
9it [01:29, 10.14s/it]
10it [01:39, 10.11s/it]
11it [01:50, 10.18s/it]
12it [02:00, 10.28s/it]
13it [02:10, 10.17s/it]
14it [02:20, 10.07s/it]
15it [02:30, 10.02s/it]
16it [02:40,  9.97s/it]
17it [02:49,  9.91s/it]
18it [02:59,  9.86s/it]
19it [03:09,  9.85s/it]
20it [03:19,  9.81s/it]
21it [03:28,  9.81s/it]
22it [03:38,  9.83s/it]
23it [03:48,  9.85s/it]
24it [03:58,  9.83s/it]


### Report

In [14]:
print("------------ Report -------------------\n")
print("Total of Analyzed Hyperparameters Combinations: {}".format(len(hyperparams)))

print("\nBest Model Hyperparameters Combination Found:\n")            

row_idx = results_df['model_dump'][results_df.recall == results_df.recall.max()].index[0]
best_model = pickle.load(open(results_df.model_dump.iloc[row_idx], 'rb'))
evalu = pickle.load(open(results_df.evaluator_dump.iloc[row_idx], 'rb'))
evalu.evaluate_model(verbose=True)

#print("\nPlot Precision vs Recall - Best Model")
#evalu.plot_precision_vs_recall()

#print("\nHeatmap of All Models")
#plot_heatmap(results_df)

#evalu.save_log()

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 24

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of WordVec_Based_Model_6': 0.3792517006802721,
              'Mean Precision of WordVec_Based_Model_6': 0.26904761904761915,
              'Mean Recall of WordVec_Based_Model_6': 0.8690476190476192},
 'Setup': [{'Name': 'WordVec_Based_Model_6'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'Tokenizer': <__main__.WordNetBased_LemmaTokenizer object at 0x7fef68a33400>}]}


### Save Similarity Matrix

In [15]:
best_model.save_sim_matrix()

#### Best Model for TOP 3 and 5 - Cosine

In [57]:
for top in [3,5]:
    row_idx_top = results_df[(results_df.top_value == top) & (results_df.metric == SimilarityMeasure.COSINE.value)].recall.argmax()
    best_model_top = pickle.load(open(results_df.model_dump.iloc[row_idx_top], 'rb'))
    evalu_top = pickle.load(open(results_df.evaluator_dump.iloc[row_idx_top], 'rb'))
    evalu_top.evaluate_model(verbose=True)
    print("------------------------------------------------------------------")

{'Measures': {'Mean FScore of WordVec_Based_Model_0': 0.4119047619047619,
              'Mean Precision of WordVec_Based_Model_0': 0.32142857142857145,
              'Mean Recall of WordVec_Based_Model_0': 0.6547619047619048},
 'Setup': [{'Name': 'WordVec_Based_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 3},
           {'Tokenizer': <__main__.PorterStemmerBased_Tokenizer object at 0x7fd9f80f9be0>}]}
------------------------------------------------------------------
{'Measures': {'Mean FScore of WordVec_Based_Model_6': 0.3792517006802721,
              'Mean Precision of WordVec_Based_Model_6': 0.26904761904761915,
              'Mean Recall of WordVec_Based_Model_6': 0.8690476190476192},
 'Setup': [{'Name': 'WordVec_Based_Model_6'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'Tokenizer': <__main__.WordNetBased_LemmaTokenizer object at 0x7fd987aa70f

### Plot Highlights

In [58]:
highlight_df(best_model.get_trace_links_df())

artf_name,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,1,0,1,1,0,1,1,1,1,1,1,1,1,1
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_010_TRG,0,1,1,1,1,0,0,0,0,1,0,0,0,1
UC_002_TRG,1,0,0,0,0,1,1,1,1,1,1,1,1,0
UC_006_TRG,1,1,0,1,1,1,1,1,0,0,0,1,0,1
UC_004_TRG,0,0,1,0,0,1,1,1,1,0,1,1,1,0
UC_005_TRG,1,0,0,0,1,0,1,1,0,0,0,1,0,0
UC_008_TRG,0,0,0,0,0,0,0,0,0,0,0,0,1,0
UC_001_TRG,1,0,1,1,0,1,0,0,1,1,1,0,1,1


In [59]:
highlight_df(orc.oracle)

,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,1,0,0,0
UC_010_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0
UC_002_TRG,0,0,0,0,1,0,0,1,0,0,0,0,0,0
UC_006_TRG,1,1,0,1,1,0,0,0,0,0,0,1,0,1
UC_004_TRG,0,0,0,0,0,1,0,0,0,0,0,0,0,0
UC_005_TRG,1,1,0,0,1,0,1,0,0,0,0,1,0,0
UC_008_TRG,0,0,0,0,0,0,0,0,0,0,0,0,1,0
UC_001_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
